# 14장 - 텐서플로의 구조 자세히 알아보기 (part 3)

In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [5]:
print(tf.__version__)
print(np.__version__)

2.5.0
1.19.5


## 추정기를 사용하여 MNIST 손글씨 숫자 분류

분류 문제를 위해 텐서플로에서 제공하는 DNNClassifier 추정기를 사용한다.  
이 추정기를 사용하면 다층 퍼셉트론을 매우 간단하게 구현할 수 있다. 이전 절에서 사전에 준비된 추정기를 사용하는 네 단계를 자세히 설명했다. 여기서도 이를 반복한다.  
첫 번째 tensorflow_datasets(tdfs) 모듈을 임포트하여 MNIST 데이터셋을 로드하고 모델의 하이퍼파라미터를 지정한다.   

설정 단계는 데이터셋 로드와 하이퍼파라미터(데이터를 섞기 위한 BUFFER_SIZE, 미니 배치 크기인 BATCH_SIZE, 훈련 에포크 횟수) 설정을 포함한다.

In [6]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 20
steps_per_epoch = np.ceil(60000 / BATCH_SIZE)

`steps_per_epoch`는 하나의 에포크에서 반복할 횟수를 결정한다. 이는 무한히 반복하는 데이터셋을 사용하기 때문에 필요하다. 그다음 입력 이미지와 레이블을 전처리하는 헬퍼 함수를 정의한다.  

입력 이미지는 원래 ([0, 255] 범위인) `'unit8'` 타입이기 때문에 `tf.image.convert_image_dtype()`을 사용하여 `tf.float32` 타입으로 변환한다. 결국([0, 1] 범위가 된다)

In [7]:
def preprocess(item):
    image = item['image']
    label = item['label']
    image = tf.image.convert_image_dtype(
        image, tf.float32)
    image = tf.reshape(image, (-1,))

    return {'image-pixels':image}, label[..., tf.newaxis]

In [8]:
# 단계 1: 입력 함수 두 개를 정의합니다(하나는 훈련용이고 다른 하나는 평가용입니다).
## 훈련용 입력 함수
def train_input_fn():
    datasets = tfds.load(name='mnist')
    mnist_train = datasets['train']

    dataset = mnist_train.map(preprocess)
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset.repeat()

## 평가용 입력 함수
def eval_input_fn():
    datasets = tfds.load(name='mnist')
    mnist_test = datasets['test']
    dataset = mnist_test.map(preprocess).batch(BATCH_SIZE)
    return dataset

In [9]:
## 단계 2: 특성 열을 정의합니다.
image_feature_column = tf.feature_column.numeric_column(
    key='image-pixels', shape=(28*28))

크기가 784(즉, 28X28)인 특성 열을 정의했다. 이 크기는 MNIST 이미지를 일렬로 펼친 것이다. 

In [11]:
## 단계 3: 추정기 객체를 만듭니다.
dnn_classifier = tf.estimator.DNNClassifier(
    feature_columns=[image_feature_column],
    hidden_units=[32, 16],
    n_classes=10,
    model_dir='models/mnist-dnn/')


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/mnist-dnn/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
## 단계 4: 훈련합니다.
dnn_classifier.train(
    input_fn=train_input_fn,
    steps=NUM_EPOCHS * steps_per_epoch)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 18760 into models/mnist-dnn/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 18760...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 18760...


INFO:tensorflow:Loss for final step: 0.25858837.


INFO:tensorflow:Loss for final step: 0.25858837.


In [13]:
eval_result = dnn_classifier.evaluate(
    input_fn=eval_input_fn)

print(eval_result)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2022-11-23T14:19:05


...

INFO:tensorflow:Saving dict for global step 18760: accuracy = 0.8978, average_loss = 0.39362174, global_step = 18760, loss = 0.3936168


INFO:tensorflow:Saving dict for global step 18760: accuracy = 0.8978, average_loss = 0.39362174, global_step = 18760, loss = 0.3936168


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 18760: models/mnist-dnn/model.ckpt-18760


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 18760: models/mnist-dnn/model.ckpt-18760


{'accuracy': 0.8978, 'average_loss': 0.39362174, 'loss': 0.3936168, 'global_step': 18760}


지금까지 사전에 준비된 추정기를 사용하는 방법을 배웠다. 또한, 기존 모델이 특정 문제에 적합한지 미리 평가하기 위해 추정기를 적용하는 방법을 알아보았다.  
사전에 준비된 추정기를 사용하는 것 말고도 케라스 모델을 추정기로 변환할 수 있다. 

## 케라스 모델에서 추정기 만들기

케라스 모델을 추정기로 바꾸는 것은 학계는 물론 산업 분야에서도 유용하다. 예를 들어 개발한 모델을 배포하거나 좆직 내 다른 구성원에게 모델을 공유할 수 있다.  
추정기로 변환하면 분산 훈련이나 체크포인트 자동 저장과 같은 장점을 활용할 수 있다. 또한, 모델을 사용하기 쉽다.  
특성 열과 입력 함수를 지정하여 입력 특성을 해석하기 때문에 혼등을 피할 수 있다.  

케라스 모델에서 추정기를 만드는 방법을 이전의 XOR 문제를 사용하여 알아보겠다. 먼저 데이터를 다시 생성하고 이를 훈련 데이터셋과 검증 데이터셋으로 나눈다. 

In [14]:
## 결과 재현을 위한 랜덤 시드
tf.random.set_seed(1)
np.random.seed(1)

## 데이터 생성
x = np.random.uniform(low=-1, high=1, size=(200, 2))
y = np.ones(len(x))
y[x[:, 0] * x[:, 1]<0] = 0

x_train = x[:100, :]
y_train = y[:100]
x_valid = x[100:, :]
y_valid = y[100:]

나중에 추정기로 변환할 케라스 모델도 만든다. 이전처럼 `Sequential` 클래스를 사용하여 모델을 정의한다. 이번에는 모델 입력에 이름을 부여하기 위해 `tf.keras.layers.Input`으로 입력층을 추가한다. 

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,), name='input-features'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


그다음 이전 절에서 설명한 네 단계를 진행한다.  

In [16]:
## 단계 1: 입력 함수를 정의합니다.
def train_input_fn(x_train, y_train, batch_size=8):
    dataset = tf.data.Dataset.from_tensor_slices(
        ({'input-features':x_train}, y_train.reshape(-1, 1)))

    # 셔플, 반복, 배치
    return dataset.shuffle(100).repeat().batch(batch_size)

def eval_input_fn(x_test, y_test=None, batch_size=8):
    if y_test is None:
        dataset = tf.data.Dataset.from_tensor_slices(
            {'input-features':x_test})
    else:
        dataset = tf.data.Dataset.from_tensor_slices(
            ({'input-features':x_test}, y_test.reshape(-1, 1)))


    # 셔플, 반복, 배치
    return dataset.batch(batch_size)

In [17]:
## 단계 2: 특성 열을 정의합니다.
features = [
    tf.feature_column.numeric_column(
        key='input-features:', shape=(2,))
]
    
features

[NumericColumn(key='input-features:', shape=(2,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

3단계에서 사전에 준비된 추정기의 객체를 만드는 대신 `tf.keras.estimator.model_to_estimator`로 이 모델을 추정기로 변환한다. 모델을 변환하기 전에 먼저 모델을 컴파일해야 한다. 

In [18]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

my_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model,
    model_dir='models/estimator-for-XOR/')

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using the Keras model provided.


INFO:tensorflow:Using the Keras model provided.


INFO:tensorflow:Using config: {'_model_dir': 'models/estimator-for-XOR/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


/Users/hanhyeongu/miniforge3/envs/hg/lib/python3.8/site-packages/tensorflow/python/keras/backend.py:435: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
2022-11-23 14:28:17.062639: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-23 14:28:17.062659: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-11-23 14:28:17.071187: I tensorflow/core/grappler/optimizers/custom_graph_opti

In [19]:
## 단계 4: 추정기를 사용합니다: 훈련/평가/예측

num_epochs = 200
batch_size = 2
steps_per_epoch = np.ceil(len(x_train) / batch_size)

my_estimator.train(
    input_fn=lambda: train_input_fn(x_train, y_train, batch_size),
    steps=num_epochs * steps_per_epoch)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='models/estimator-for-XOR/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})


...

INFO:tensorflow:Saving checkpoints for 10000 into models/estimator-for-XOR/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 10000...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 10000...


INFO:tensorflow:Loss for final step: 0.0016787027.


INFO:tensorflow:Loss for final step: 0.0016787027.


In [20]:
my_estimator.evaluate(
    input_fn=lambda: eval_input_fn(x_valid, y_valid, batch_size))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


/Users/hanhyeongu/miniforge3/envs/hg/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2022-11-23T14:29:52


...

INFO:tensorflow:Saving dict for global step 10000: binary_accuracy = 0.95, global_step = 10000, loss = 0.084949404


INFO:tensorflow:Saving dict for global step 10000: binary_accuracy = 0.95, global_step = 10000, loss = 0.084949404


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: models/estimator-for-XOR/model.ckpt-10000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: models/estimator-for-XOR/model.ckpt-10000


{'binary_accuracy': 0.95, 'loss': 0.084949404, 'global_step': 10000}